In [1]:

import argparse



import os
from sklearn.utils import shuffle
from tqdm import tqdm
class_dirs = os.listdir("dataset")
class_dirs.sort()
print(class_dirs)
import numpy as np

hinge_feature_vectors = []
cold_feature_vectors = []
labels = []
label_names = []
ecount = 0
input_folder="dataset"
imgs=[]
import numpy as np
import cv2


# cold = Cold()
# hinge = Hinge()
for i, class_dir in enumerate(class_dirs):
    img_filenames = os.listdir(os.path.join(input_folder, class_dir))
    
    for img_filename in tqdm(img_filenames):
        
            img_path = os.path.join(input_folder, class_dir, img_filename)
            
            
            
            imgs.append(cv2.imread("dataset/"+class_dir+'/'+img_filename))
            
            label_names.append(class_dir)
            labels.append(i)

greyimgs=[]
print("------------------------------")
m=100000000000
minidx=0
k=0
for i in imgs:
     
    if i.shape[0]*i.shape[1]<m:
        m=i.shape[0]*i.shape[1]
        minidx= k
    k=k+1

print(imgs[minidx].shape)
     
print("------------------------------")






    






['flooded', 'non-flooded']


100%|██████████| 461/461 [06:04<00:00,  1.27it/s]


------------------------------
(152, 123, 3)
------------------------------


In [2]:
resizedimgs=[]
for i in imgs:
   resizedimgs.append( cv2.resize(i, (370,370)))

print("wdwdc")
print(resizedimgs[11].shape)

for i in resizedimgs:
      greyimgs.append(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY))

wdwdc
(370, 370, 3)


In [4]:
resizedimgs, label_names = shuffle(resizedimgs, label_names)
print(label_names[0:10])
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(label_names)
category_encode = le.transform(label_names)
print(category_encode[0:10])

['non-flooded', 'non-flooded', 'non-flooded', 'flooded', 'non-flooded', 'flooded', 'flooded', 'flooded', 'flooded', 'non-flooded']
[1 1 1 0 1 0 0 0 0 1]


In [5]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

import pandas as pd
import numpy as np

from PIL import Image

from skimage.feature import hog
from skimage.color import rgb2gray

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc

In [6]:
def create_features(img):
    # flatten three channel color image
    
    color_features = img.flatten()
    # convert image to greyscale
    grey_image = rgb2gray(img)
    # get HOG features from greyscale image
    hog_features = hog(grey_image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    # combine color and hog features into a single array
    flat_features = np.hstack((hog_features))
    return flat_features
bombus_features = create_features(resizedimgs[0])
print(bombus_features)

[0.19477274 0.0289264  0.02716208 ... 0.06230081 0.12333053 0.11445857]


In [7]:
def create_feature_matrix(imgs):
    features_list = []
    
    for img_id in imgs:
        # load image
        
        # get features for image
        image_features = create_features(img_id)
        features_list.append(image_features)
        
    # convert list of arrays into a matrix
    feature_matrix = np.array(features_list)
    return feature_matrix

# run create_feature_matrix on our dataframe of images
feature_matrix = create_feature_matrix(resizedimgs)
print(feature_matrix.shape)

(922, 35721)


In [8]:
# define standard scaler
ss = StandardScaler()
# run this on our feature matrix
bees_stand = ss.fit_transform(feature_matrix)

pca = PCA(n_components=500)
# use fit_transform to run PCA on our standardized matrix
bees_pca = ss.fit_transform(bees_stand)
# look at new shape
print('PCA matrix shape is: ', bees_pca.shape)

PCA matrix shape is:  (922, 35721)


In [28]:
X = pd.DataFrame(bees_pca)
y = pd.Series(label_names)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.2,
                                                    random_state=1234)

# look at the distrubution of labels in the train set
pd.Series(y_train).value_counts()

flooded        372
non-flooded    365
dtype: int64

In [29]:
# define support vector classifier
svm = SVC(kernel='linear', probability=True, random_state=123)

# fit model
svm.fit(X_train, y_train)

SVC(kernel='linear', probability=True, random_state=123)

In [30]:
from sklearn.metrics import classification_report
print(len(X_test))
y_pred = svm.predict(X_test)

# calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print('Model accuracy is: ', accuracy)
print(classification_report(y_test,y_pred))

185
Model accuracy is:  0.8378378378378378
              precision    recall  f1-score   support

     flooded       0.86      0.80      0.83        89
 non-flooded       0.82      0.88      0.85        96

    accuracy                           0.84       185
   macro avg       0.84      0.84      0.84       185
weighted avg       0.84      0.84      0.84       185



In [40]:
X = pd.DataFrame(bees_pca)
y = pd.Series(category_encode)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.2,
                                                    random_state=1234)

# look at the distrubution of labels in the train set
pd.Series(y_train).value_counts()

0    372
1    365
dtype: int64

In [42]:
from sklearn.linear_model import LogisticRegression


classifier = LogisticRegression()
classifier.fit(X_train, y_train)


res=classifier.predict(X_test)
accuracy = accuracy_score(y_test, res)
print(accuracy)

0.8432432432432433


In [43]:
svm = SVC(kernel='linear', probability=True, random_state=123)

# fit model
svm.fit(X_train, y_train)

SVC(kernel='linear', probability=True, random_state=123)

In [45]:
# use category encoding
from sklearn.metrics import classification_report
print(len(X_test))
y_pred = svm.predict(X_test)

# calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print('Model accuracy is: ', accuracy)
print(classification_report(y_test,y_pred))

185
Model accuracy is:  0.8378378378378378
              precision    recall  f1-score   support

           0       0.86      0.80      0.83        89
           1       0.82      0.88      0.85        96

    accuracy                           0.84       185
   macro avg       0.84      0.84      0.84       185
weighted avg       0.84      0.84      0.84       185



In [46]:
from sklearn.naive_bayes import GaussianNB


# instantiate the model
gnb = GaussianNB()


# fit the model
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score: 0.7946


In [47]:
# Importing all necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = 370, 370


In [48]:
if K.image_data_format() == 'channels_first':
    	input_shape = (3, img_width, img_height)
else:
	input_shape = (img_width, img_height, 3)


In [62]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [63]:
model.compile(loss='binary_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])


In [55]:
print(np.array( resizedimgs).shape)
resizedimgs=np.array( resizedimgs)

(922, 370, 370, 3)


In [64]:
X_train=resizedimgs.reshape(resizedimgs.shape[0],370,370,3)
# X_test=X_test.reshape(X_test.shape[0],28,28,1)

In [ ]:
model.fit(X_train, np.array(category_encode), batch_size=256, epochs=1)